# Experiment 0605 - can module survive high temperature

In [ ]:
# dependency
%pip install python-dotenv
%pip install openai
%pip install pandas

In [1]:
import os
from dotenv import load_dotenv, find_dotenv # load keys from .env file
import openai # use OpenAI API
import random
import re
import json
import pandas as pd
import numpy as np
import datetime
import tiktoken

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')
encoding = tiktoken.get_encoding("cl100k_base")


In [2]:

encoding = tiktoken.encoding_for_model("gpt-4-0314")

def num_tokens_from_messages(messages, model="gpt-4-0314"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo":
        print("Warning: gpt-3.5-turbo may change over time. Returning num tokens assuming gpt-3.5-turbo-0301.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301")
    elif model == "gpt-4":
        print("Warning: gpt-4 may change over time. Returning num tokens assuming gpt-4-0314.")
        return num_tokens_from_messages(messages, model="gpt-4-0314")
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif model == "gpt-4-0314":
        tokens_per_message = 3
        tokens_per_name = 1
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [5]:
# search for the json object at the end using regular expressions
def get_json(text):
    '''extract the json object from the text using regular expressions'''
    match = re.search(r'{.*}', text)
    if match:
        json_string = match.group()
        # json_object = json.loads(json_string)
        return json_string

def get_python(text):
    '''extract the python code from the text using regular expressions'''
    pattern = re.compile(r'"""python(.*?)"""', re.DOTALL)
    match = pattern.search(text)
    if match:
        code = match.group(1)
        return code.strip()
    else:
        return None

def get_completion(prompt, model="gpt-3.5-turbo"):
    '''Query OpenAI API to get a completion from a naive prompt'''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=500, # this is the maximum number of returned tokens
    )
    print(">>>>RESPONSE>>>>")
    print(str(response.choices[0].message["content"]))
    print("<<<<END<<<<")
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=1.5,max_tokens=2000):
    '''ChatML format to query OpenAI API to get a completion from a list of messages'''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

# write function to combine two messages
# turn the above process into a function
def get_answer_step_by_step(question, model="gpt-4", get_completion_from_messages=get_completion_from_messages):
    '''Query OpenAI API to answer a question step-by-step'''
    system_message = """A user will give you a problem. Follow these steps.

Step 1: Reason step-by-step to clearly define the problem and what additional information is desired.

Step 2: Reason step-by-step a methodology that can be used to answer the problem.

Step 3: Calculate the results based on step 2.

Step 4: Stating assumptions used in the methodology and the results.

    """
    messages = [{"role": "system", "content": system_message},
    {"role": "user", "content": question}]
    result = get_completion_from_messages(messages, model=model) # returns a string
    return result

def text_to_json(message, model="gpt-3.5"):
    '''Query OpenAI API to convert text to a parsable JSON'''
    JSON_Parser_system_message = '''\
A user will give you a text contain 4 steps. Your task is to convert the text to make it more readable. Each step will be the key, and the details will be the value.
Your output will be a JSON object in deliminator (""") looks like this: 
JSON={"Step 1": "details in step 1", "Step 2": "details in step 2", "Step 3": "details in step 3", "Step 4": "details in step 4"}
Here's an example:
INPUT:
======
Step 1: 
To answer this problem, we need to know the following information:
1. How much power is required to jump start a car?
2. How much power can be generated by one lemon battery?
3. How many lemon batteries are required to generate the power needed to jump start a car?

Step 2:
To answer this problem, we can follow these steps:
1. Determine the power required to jump start a car.
2. Determine the power generated by one lemon battery.
3. Calculate the number of lemon batteries required to generate the power needed to jump start a car.

Step 3:
1. According to AAA, the power required to jump start a car is around 300-400 amps for 5-10 seconds. Let's assume that we need 350 amps for 7 seconds to jump start a car.
2. One lemon battery can generate around 0.7 volts and 0.0003 amps of current. Using Ohm's law, we can calculate the power generated by one lemon battery as follows: Power = Voltage x Current = 0.7 x 0.0003 = 0.00021 watts or 0.21 milliwatts.
3. To generate 350 amps for 7 seconds, we need a total power of 350 x 7 = 2450 watts or 2.45 kilowatts. To generate this much power using lemon batteries, we need 2.45 / 0.00021 = 11,666.67 lemon batteries.

Step 4:
Assumptions:
1. The power required to jump start a car is assumed to be 350 amps for 7 seconds.
2. The power generated by one lemon battery is assumed to be 0.21 milliwatts.
3. The lemon batteries are assumed to be connected in series to generate the required voltage and current.

======
OUTPUT:"""
JSON={"problem definition": "If a lemon can be made into a battery, how many lemons can jump start a car?", "methodology": "To answer this problem, we need to know the power required to jump start a car, the power generated by one lemon battery, and the number of lemon batteries required to generate the power needed to jump start a car. We can calculate the number of lemon batteries required by dividing the required power by the power generated by one lemon battery.", "results": "To jump start a car, we need 11,666.67 lemon batteries.", "assumptions": "The power required to jump start a car is assumed to be 350 amps for 7 seconds. The power generated by one lemon battery is assumed to be 0.21 milliwatts. The lemon batteries are assumed to be connected in series to generate the required voltage and current."}
"""
'''
    message_wrapper = f'''Convert message into JSON object.\nInput:\n==={message}\n===\n OUTPUT:\n """[JSON=]"""'''
    get_json_messages = [{"role": "system", "content": JSON_Parser_system_message},
    {"role": "user", "content": message_wrapper}]

    JSON_result = get_completion_from_messages(get_json_messages) # returns a string
    return JSON_result

def text_to_python(message, model="gpt-4"):
    '''Query OpenAI API to convert text to python'''
    Python_generator_system_message = '''A user will give you a text contain 4 steps. Your task is to edit the text to make it more readable in the following format.
Output a JSON object after deliminator """ in the following structure: JSON={"Step 1": "details in step 1", "Step 2": "details in step 2", "Step 3": "details in step 3", "Step 4": "details in step 4"}
One Example is as follows:
INPUT TEXT:
Step 1: 
To answer this problem, we need to know the following information:
1. How much power is required to jump start a car?
2. How much power can be generated by one lemon battery?
3. How many lemon batteries are required to generate the power needed to jump start a car?

Step 2:
To answer this problem, we can follow these steps:
1. Determine the power required to jump start a car.
2. Determine the power generated by one lemon battery.
3. Calculate the number of lemon batteries required to generate the power needed to jump start a car.

Step 3:
1. According to AAA, the power required to jump start a car is around 300-400 amps for 5-10 seconds. Let's assume that we need 350 amps for 7 seconds to jump start a car.
2. One lemon battery can generate around 0.7 volts and 0.0003 amps of current. Using Ohm's law, we can calculate the power generated by one lemon battery as follows: Power = Voltage x Current = 0.7 x 0.0003 = 0.00021 watts or 0.21 milliwatts.
3. To generate 350 amps for 7 seconds, we need a total power of 350 x 7 = 2450 watts or 2.45 kilowatts. To generate this much power using lemon batteries, we need 2.45 / 0.00021 = 11,666.67 lemon batteries.

Step 4:
Assumptions:
1. The power required to jump start a car is assumed to be 350 amps for 7 seconds.
2. The power generated by one lemon battery is assumed to be 0.21 milliwatts.
3. The lemon batteries are assumed to be connected in series to generate the required voltage and current.

OUTPUT PYTHON:
"""python
    # Step 1: Define the variables
    power_car_jump_start_amps = 350    # Amps
    time_car_jump_start_seconds = 7    # Seconds

    lemon_battery_volts = 0.7          # Volts
    lemon_battery_amps = 0.0003        # Amps

    # Step 2: Calculate the power required to jump start a car and the power generated by one lemon battery

    # Power = Voltage x Current
    power_car_jump_start_watts = power_car_jump_start_amps * time_car_jump_start_seconds  # Watts
    power_lemon_battery_watts = lemon_battery_volts * lemon_battery_amps  # Watts

    # Step 3: Calculate the number of lemon batteries required to generate the power needed to jump start a car
    number_of_lemon_batteries = power_car_jump_start_watts / power_lemon_battery_watts

    # Step 4: Print the result
    print(f"You need approximately {number_of_lemon_batteries:.0f} lemon batteries to jump start a car.")
"""
Your will only output the python code after """ in the following format:
"""python
# Step 1: Define the variables
# ...
# Last step: Print the result
print(f"...")
"""
'''
    get_python_messages = [{"role": "system", "content": Python_generator_system_message},
    {"role": "user", "content": message}]
    result = get_completion_from_messages(get_python_messages, model=model) # returns a string
    return result
# # Example of an OpenAI ChatCompletion request with stream=True
# # https://platform.openai.com/docs/guides/chat

# # a ChatCompletion request

In [ ]:
# Uncomment and Run this code block to get an example output

# prompt_example= f"""What is Fermi Estimation and how does it work?"""
# response = get_completion(prompt_example)


In [6]:
# market sizing question generator
# list of service providers
service_providers = ["piano tuners", "nurseries", "coffee shops", "restaurants", "libraries", "gyms", "hotels", "barber shops", "gas stations", "schools"]

# list of cities
cities = ["Chicago", "London", "Beijing", "New York", "Los Angeles", "Paris", "Berlin", "Tokyo", "Sydney", "Mumbai"]

# function to generate a question
def generate_market_sizing_question():
    service_provider = random.choice(service_providers)
    city = random.choice(cities)
    
    question = f"How many {service_provider} are there in {city}?"
    
    return question

# generate a question
print(generate_market_sizing_question())

How many libraries are there in London?


In [7]:
# warm up talk
# turn the above process into a function
you_are_fermi = "Your name is Enrico Fermi. You are an expert physicist and mathematician. You are known for your ability to make good approximate calculations with little or no actual data. You are also known for your ability to solve problems using recursion."
round_1_fermi = '''Hi Fermi, are you alright? I want to learn how to use your trick to do Fermi Estimation and improve my problem solving skills.'''
round_1_response = '''Fermi estimation, also known as Fermi problems, is a way to make good approximate calculations with little or no actual data. The method is named after physicist Enrico Fermi as he was known for making good approximate calculations with little or no actual data.

Fermi problems typically involve making reasonable assumptions about the problem, then performing simple calculations on those assumptions. Here's a step-by-step guide on how to apply this:

1. Define the Problem Clearly: Start by identifying the problem you need to solve. Fermi problems often seem overwhelming at first glance due to the large numbers and broad scope involved. For example, "How many piano tuners are there in New York City?".

2. Break Down the Problem: Next, you'll want to break down the problem into smaller, more manageable questions whose answers can be reasonably estimated. For the piano tuner question, you might ask:

- How many people live in New York City?
- How many households does that represent?
- What fraction of households own a piano?
- How often does a piano need to be tuned each year?
- How many pianos can one tuner handle in a year?
3. Make Estimates: Now, you'll need to make educated guesses to answer these smaller questions. You can often make a reasonable estimate based on known facts or basic assumptions about the world. For example, you might guess that one in every hundred households in New York City owns a piano, and that a piano needs to be tuned once a year.

4. Perform Calculations: Once you have your estimates, you'll need to perform the necessary calculations to solve the problem. Remember, the goal here isn't to get a perfectly accurate answer, but rather to arrive at a ballpark figure that's within the right order of magnitude. So, if your calculations require complex operations, it's usually okay to round numbers or simplify to make the math easier.

5. Review and Refine: Finally, you should always review your estimates and calculations to see if they make sense. If your answer seems unreasonable, you should go back and check your work, refine your estimates, or consider whether there are other aspects of the problem you haven't taken into account.

Fermi estimation is an incredibly valuable tool for developing problem-solving skills and enhancing critical thinking. It encourages making reasonable assumptions, simplifying complex problems, and approximating – all crucial skills in a wide range of fields.'''

round_2_recursion = '''This is a very structured approach. I am interested in the second step, which is breaking down a problem into sub-problems. This is a recursive step. Can you explain the concept of recursion?'''
round_2_response = '''Recursion is a fundamental concept in computer science, mathematics, and broader problem-solving that involves breaking a complex problem into smaller, manageable subproblems. The solution to the original problem is then constructed from the solutions of these subproblems.

A recursive process has two key components: a base case and a recursive case. The base case is a condition that terminates the recursion, providing a direct solution without self-referencing. In contrast, the recursive case involves the function calling itself with smaller instances of the problem, leading towards the base case with each step.

For example, a recursive function for calculating factorials in programming would include a base case (if n equals 0, return 1) and a recursive case (return n multiplied by the factorial of n-1). This function continually calls itself, moving closer to the base case each time, until it can start returning values.

In Fermi estimation and similar problem-solving methods, recursion entails breaking the problem down into subproblems until they can be easily solved or estimated. This 'divide and conquer' approach helps navigate complex problems, with the solutions to subproblems combined to solve larger ones, eventually addressing the original problem.'''

round_3_piano_tuners_explained = '''In the case of the classic piano tuner problem, would you explain how recursion is applied here?'''
round_3_response = '''In the case of the piano tuner problem, the application of recursion isn't as pronounced as in a mathematical function or computer algorithm, but the concept of "breaking down a problem into sub-problems until they are solvable" is present, which aligns with the general idea of recursion.

The original problem is: "How many piano tuners are there in New York City?" This is a difficult problem to answer directly due to the lack of readily available data, so we break it down into smaller problems. Here's how that might look:

Estimate the population of New York City: This is a base problem we can solve with known data. As of my knowledge cutoff in 2021, the population of New York City was over 8 million people.

Estimate the number of households in New York City: This is a smaller problem we can estimate with a reasonable assumption. For instance, we could assume an average of 2.5 people per household, which is roughly the average household size in the United States.

Estimate the proportion of households with a piano: This is another smaller problem. We might not have an exact answer, but we can make a reasonable guess. Maybe 1 in every 50 households has a piano.

Estimate the number of times a piano needs tuning per year: Yet another smaller problem. A piano might need to be tuned once or twice a year.

Estimate the number of pianos a tuner can service in a year: This is the final smaller problem. A piano tuner might be able to tune 2 pianos a day, working 5 days a week, for 50 weeks a year.

Each of these smaller problems contributes to the solution of the larger problem. Once we have estimates for each of these, we can combine them to find the total estimated number of piano tuners in New York City. This process is similar to recursion in that we're continually breaking down the problem into smaller, solvable problems until we're able to solve the original, more complex problem.'''

counter = 0
question_bank = []
while counter < 10:
    # check if the question is already in the bank
    new_question = generate_market_sizing_question()
    if new_question not in question_bank:
            question_bank.append(new_question)
            counter += 1
print(question_bank)

['How many restaurants are there in Paris?', 'How many schools are there in Beijing?', 'How many hotels are there in Los Angeles?', 'How many coffee shops are there in Beijing?', 'How many nurseries are there in New York?', 'How many piano tuners are there in Tokyo?', 'How many gas stations are there in Sydney?', 'How many libraries are there in Tokyo?', 'How many hotels are there in Paris?', 'How many barber shops are there in Beijing?']


In [8]:
def apply_concept_recursion(chat_messages, model="gpt-4-0314", get_completion_from_messages=get_completion_from_messages):
    '''Query OpenAI API to answer a question using recursion'''
    result = get_completion_from_messages(chat_messages, model=model) # returns a string
    return result

## Question inference (with GPT4 model)

In [12]:
# generate a question bank of 100 questions
df = pd.DataFrame(columns=['question', 'answer_after_warm_up', 'answer_json', 'answer_python'])
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
for index, each_question in enumerate(question_bank):
    question_prompt = f'''That's very helpful. Now let's consider another problem: {each_question} - how would you apply the concept of recursion to arrive at an answer?
Can you use your estimation trick and recursively break the problem down until you can confidently provide a rough estimation?'''
    warmed_messages = [{"role": "system", "content": you_are_fermi},
            {"role": "user", "content": round_1_fermi},
            {"role": "system", "content": round_1_response},
            {"role": "user", "content": round_2_recursion},
            {"role": "system", "content": round_2_response},
            {"role": "user", "content": round_3_piano_tuners_explained},
            {"role": "system", "content": round_3_response},
            {"role": "user", "content": question_prompt}
            ]
    print(each_question)
    prompt_token_count = num_tokens_from_messages(warmed_messages)
    print(f"""No. of token used: {prompt_token_count}""")
    try:
        answer = apply_concept_recursion(warmed_messages, get_completion_from_messages(warmed_messages,max_tokens=4096-2*prompt_token_count))
        print(answer)
    except Exception as e:
        answer = '''Error'''
        answer_json = {"error": e}
        python_message = '''"""python\nstatus = "error"\nprint(stats)"""'''
        print(e)
        row = [each_question, answer, answer_json, python_message]
        df.loc[len(df)] = row
        continue # next loop
    print(f"<<<<Completed Question {index} in question bank<<<<")
    try:
        answer_json = text_to_json(answer)
    except Exception as e:
        answer_json = {"error": e}
        print(e)
    print(f"<<<<Received Response to Question {index} <<<<")
    try:
        python_message = text_to_python(answer)
    except Exception as e:
        python_message = '''"""python\nstatus = "error"\nprint(stats)"""'''
        print(e)
    print(f"<<<<Received Python Code to Question {index} <<<<")
    # append the new data to the end of the dataframe
    row = [each_question, answer, answer_json, python_message]
    df.loc[len(df)] = row

df.to_csv(f'market_sizing_high_temp_{timestamp}.csv', index=False)
print(question_bank)
print(answer)
print(get_json(answer_json))
print(get_python(python_message))

## get_answer_step_by_step is about 35s per question
## get_json is about 15s per question
## get_python is about 25s per question
# total = 75s per question


How many restaurants are there in Paris?
No. of token used: 1357
internal error {
    "error": {
        "message": "internal error",
        "type": "invalid_request_error",
        "param": null,
        "code": null
    }
}
 500 {'error': {'message': 'internal error', 'type': 'invalid_request_error', 'param': None, 'code': None}} {'Date': 'Mon, 05 Jun 2023 08:13:48 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '147', 'Connection': 'keep-alive', 'vary': 'Origin', 'x-request-id': '7f33758efd36bcf1d882b3ce5cad17a8', 'strict-transport-security': 'max-age=15724800; includeSubDomains', 'CF-Cache-Status': 'DYNAMIC', 'Server': 'cloudflare', 'CF-RAY': '7d26f53b6d2f778f-LHR', 'alt-svc': 'h3=":443"; ma=86400'}
<<<<Completed Question 0 in question bank<<<<
<<<<Received Response to Question 0 <<<<
<<<<Received Python Code to Question 0 <<<<
How many schools are there in Beijing?
No. of token used: 1357
internal error {
    "error": {
        "message": "internal erro

In [ ]:
# apply a function to the column "answer_python" for each row
# df[df['name'].apply(lambda x: is_long(x) & is_short(x))]
for index, row in df.iterrows():
    # print(row['answer_python'])
    exec(get_python(df['answer_python'][index]))



There are approximately 12600 nurseries in Tokyo.
Based on the assumptions and calculations, there are approximately 548 gas stations in Los Angeles.
Based on the Fermi estimation, there could be around 4200 schools in Beijing.
There are approximately 21600 nurseries in Mumbai.
Using Fermi estimation, there are approximately 5220 barber shops in London.
There are approximately 105 libraries in Paris.
There might be around 10500 coffee shops in Beijing.
Based on our assumptions, there are approximately 8000 coffee shops in New York City.
There could be approximately 3523 hotels in Chicago based on these assumptions and calculations.
An estimated 320 piano tuners are needed to service all the pianos in New York State.


In [30]:
df.to_csv('market_sizing_question.csv', index=False)

In [31]:
# exec code directly will be dangerous, so we use exec to run the code in a sandbox
#exec(get_python(python_message))
# just print the message will do : )



In [32]:
# Best practices:
# https://platform.openai.com/docs/guides/gpt-best-practices/strategy-split-complex-tasks-into-simpler-subtasks

# Strategy: Split complex tasks into simpler subtasks

In [33]:
# Strategy: Use a persona

In [34]:
# Strategy: User inner monologue
# instruct the model to put parts of the output that are meant to be hidden from the user into a structured format
# makes parsing them easy. 

# before presenting the output to the user, the output is parsed and only part of the output is made visible.

system_message_IM_example = '''Follow these steps to answer the user queries.

Step 1 - First work out your own solution to the problem. Don't rely on the student's solution since it may be incorrect. Enclose all your work for this step within triple quotes (""").

Step 2 - Compare your solution to the student's solution and evaluate if the student's solution is correct or not. Enclose all your work for this step within triple quotes (""").

Step 3 - If the student made a mistake, determine what hint you could give the student without giving away the answer. Enclose all your work for this step within triple quotes (""").

Step 4 - If the student made a mistake, provide the hint from the previous step to the student (outside of triple quotes). Instead of writing "Step 4 - ..." write "Hint:".'''